## NAME: SHUBHAM GOEL
## ROLL NO: 101903748
## GROUP: 3COE28

In [292]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [293]:
import cudf as pd
import cupy as cp
import cuml
from cuml import LinearRegression
import seaborn as sns
from cuml.linear_model import LinearRegression
from cuml import Ridge
from cuml.linear_model import Ridge
from cuml.model_selection import train_test_split
from cuml.linear_model import Lasso
from cuml.ensemble import RandomForestRegressor

## Importing Data:

In [294]:
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

In [295]:
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [296]:
train_df.shape,test_df.shape

In [297]:
num_train = train_df.shape[0]
num_train

In [298]:
num_test = test_df.shape[0]
num_test

In [299]:
train_df.columns

In [300]:
Y = train_df['SalePrice']

In [301]:
train_df.drop('SalePrice', axis = 1 , inplace = True)

In [302]:
train_df

### Counting missing values

In [303]:
train_df.isnull().sum()

In [304]:
test_df.isnull().sum()

### Visualization of data using heatmap 
### Calculating the number of categorical features and numerical features

In [305]:
numerical_features = train_df.dtypes[train_df.dtypes != 'object'].index
print(numerical_features)
print("Total number of numerical features = " , len(numerical_features))

In [306]:
categorical_features = train_df.dtypes[train_df.dtypes == 'object'].index
print(categorical_features)
print("Total number of categorical features = " , len(categorical_features))

### Handling the NULL values 

In [307]:
train_df['LotFrontage'].value_counts()

In [308]:
train_df['LotFrontage']= train_df['LotFrontage'].fillna(train_df['LotFrontage'].mean())
train_df['LotFrontage'].value_counts()

In [309]:
test_df['LotFrontage']= test_df['LotFrontage'].fillna(test_df['LotFrontage'].mean())
test_df['LotFrontage'].value_counts()

### Handling Categorical Features for Training Data
### Handling Categorical Features for Test Data

In [310]:
test_df['BsmtCond']=test_df['BsmtCond'].fillna(test_df['BsmtCond'].mode()[0])
test_df['BsmtQual']=test_df['BsmtQual'].fillna(test_df['BsmtQual'].mode()[0])
test_df['FireplaceQu']=test_df['FireplaceQu'].fillna(test_df['FireplaceQu'].mode()[0])
test_df['GarageType']=test_df['GarageType'].fillna(test_df['GarageType'].mode()[0])
test_df['GarageFinish']=test_df['GarageFinish'].fillna(test_df['GarageFinish'].mode()[0])
test_df['GarageQual']=test_df['GarageQual'].fillna(test_df['GarageQual'].mode()[0])
test_df['GarageCond']=test_df['GarageCond'].fillna(test_df['GarageCond'].mode()[0])
test_df['MasVnrType']=test_df['MasVnrType'].fillna(test_df['MasVnrType'].mode()[0])
test_df['MasVnrArea']=test_df['MasVnrArea'].fillna(test_df['MasVnrArea'].mode()[0])
test_df['BsmtExposure']=test_df['BsmtExposure'].fillna(test_df['BsmtExposure'].mode()[0])
test_df['BsmtFinType2']=test_df['BsmtFinType2'].fillna(test_df['BsmtFinType2'].mode()[0])
test_df['MSZoning']=test_df['MSZoning'].fillna(test_df['MSZoning'].mode()[0])
test_df['Utilities']=test_df['Utilities'].fillna(test_df['Utilities'].mode()[0])
test_df['Exterior1st']=test_df['Exterior1st'].fillna(test_df['Exterior1st'].mode()[0])
test_df['Exterior2nd']=test_df['Exterior2nd'].fillna(test_df['Exterior2nd'].mode()[0])
test_df['BsmtFinType1']=test_df['BsmtFinType1'].fillna(test_df['BsmtFinType1'].mode()[0])
test_df['SaleType']=test_df['SaleType'].fillna(test_df['SaleType'].mode()[0])
test_df['BsmtFullBath']=test_df['BsmtFullBath'].fillna(test_df['BsmtFullBath'].mode()[0])
test_df['BsmtHalfBath']=test_df['BsmtHalfBath'].fillna(test_df['BsmtHalfBath'].mode()[0])
test_df['KitchenQual']=test_df['KitchenQual'].fillna(test_df['KitchenQual'].mode()[0])
test_df['Functional']=test_df['Functional'].fillna(test_df['Functional'].mode()[0])
test_df['GarageCars']=test_df['GarageCars'].fillna(test_df['GarageCars'].mean())
test_df['GarageArea']=test_df['GarageArea'].fillna(test_df['GarageArea'].mean())
test_df['BsmtFinSF1']=test_df['BsmtFinSF1'].fillna(test_df['BsmtFinSF1'].mean())
test_df['BsmtFinSF2']=test_df['BsmtFinSF2'].fillna(test_df['BsmtFinSF2'].mean())
test_df['BsmtUnfSF']=test_df['BsmtUnfSF'].fillna(test_df['BsmtUnfSF'].mean())
test_df['TotalBsmtSF']=test_df['TotalBsmtSF'].fillna(test_df['TotalBsmtSF'].mean())

In [311]:
train_df.columns

### Dropping values that are less than 50 % 

In [312]:
train_df.drop(['Alley' , 'PoolQC' , 'Fence' , 'MiscFeature' , 'GarageYrBlt' , 'Id'] , axis = 1 , inplace = True)

In [313]:
test_df.drop(['Alley' , 'PoolQC' , 'Fence' , 'MiscFeature' , 'GarageYrBlt' , 'Id'] , axis = 1 , inplace = True)

In [314]:
train_df.shape

In [315]:
test_df.shape

### Visualizing again after drawing the heat map

In [316]:
train1 = train_df
train1 = train1.to_pandas()
sns.heatmap(train1.isnull() )

In [317]:
train_df.isnull().sum()

### Checking for remaining null values

In [318]:
train_df.isnull().any().any

In [319]:
test_df.isnull().any().any

In [320]:
train_df = train_df.fillna(method = 'pad')

In [321]:
train_df.isnull().any().any

In [322]:
final_df = pd.concat([train_df , test_df] , axis = 0)

In [323]:
final_df.shape

In [324]:
final_df.head()

In [325]:
final_df = pd.get_dummies(final_df)

In [326]:
final_df.shape

### Splitting data into train , test 

In [327]:
train_df = final_df.iloc[:num_train , :]
test_df = final_df.iloc[num_test+1 :, :]

In [328]:
train_df.shape

In [329]:
test_df.shape

In [330]:
final_df.head()

In [331]:
X = train_df
X.shape

In [332]:
Y.shape

### Model Creation - Linear Regression 

In [333]:
X_train , X_test , y_train , y_test = train_test_split(X , Y , test_size=0.29 , random_state = 42)

In [334]:
X.isnull().any().any

In [335]:
X_train.isnull().any().any

### Creating the model and fitting it 

In [336]:
linreg = LinearRegression(fit_intercept = True , normalize = False , algorithm = 'svd-jacobi')

In [337]:
split = linreg.fit(X,Y)

In [338]:
print("R-Squared Value for Training Set: {:.3f}".format(linreg.score(X_train.astype('float32'), y_train.astype('float32'))))

In [339]:
test_df.shape

### Predictions

In [340]:
y_pred_split=linreg.predict(test_df)

In [341]:
y_pred_split

In [342]:
y_pred_split.shape

### All 5 linear Regression Algorithms

In [343]:
algorithm = ['svd', 'eig', 'svd-qr', 'svd-jacobi']

### Evaluation metrics for all 5 algorithms

In [344]:
for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(X_train,y_train)
    preds = lr.predict(X_test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(y_test.astype('int64'),preds.astype('int64')))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(y_test.astype('float32'),preds.astype('float32')))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(y_test.astype('int64'),preds.astype('int64')))

In [345]:
sample_sub = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample_sub.head()

In [346]:
sample_sub['SalePrice']

In [347]:
sample_sub['SalePrice'] = y_pred_split
sample_sub.to_csv('submission.csv' , index = False)

In [348]:
sample_sub['SalePrice']

### Ridge Regression 

### Submission

In [349]:
y_pred_ridge1 = sample_sub['SalePrice']

sample_sub.to_csv('submission1.csv', index=False)

In [350]:
y_pred_ridge2 = sample_sub['SalePrice']

sample_sub.to_csv('submission2.csv', index=False)

### Lasso Regression

In [351]:
ls = Lasso(alpha = 0.1)
result_lasso = ls.fit(X, Y)
y_pred_lasso=result_lasso.predict(test_df)

In [352]:
y_pred_lasso

### Submission

In [353]:
sample_sub['SalePrice'] = y_pred_lasso

sample_sub.to_csv('submission3.csv', index=False)